In [1]:
import random
from typing import Dict, List
import MeCab
import re
import pandas as pd 
data1=pd.read_csv("/Users/withmocha/Desktop/DATA/Capston Design(2024)/data(sentence)/0518(final)/abnormal row/대출사기형(4문장).csv",index_col=0)
data2=pd.read_csv("/Users/withmocha/Desktop/DATA/Capston Design(2024)/data(sentence)/0518(final)/abnormal row/수사기관형(4문장).csv",index_col=0)

In [2]:
data1

,sentence,predict
0,여보세요 네 님 예 예 어저께 통화했던 최영환입니다,1
1,아 예 예 어떻게 가진행 결과 안내차 오늘 아침에 전화 드린다고 제가 얘기를 했었...,1
2,네 금리는 연 금리 5.5%에 상환 기간 60개월입니다. 5년이고요 네,1
3,매 월 상환하시는 금액은 60개월 기준 원금 이자 포함되신 금액으로 573035원...,1
4,금리가 아 예 예 예. 말씀하세요 괜찮으세요?,1
...,...,...
1391,아 그러니까 사장님께서 그 안쓰시는 통장 있으시면 400에서 450까지 선지급 해...,1
1392,네 알겠습니다 네네 네 여보세요 뭐 어디세요?,1
1393,아 네 안녕하세요 저희 그 하나캐피탈에서 연락을 드렸습니다 아 그래요? 네네네 고...,1
1394,지금 현재 이용중이신 고금리건 채무 러시앤캐시 산와머니 이런 고금리건 채무 이용하...,1


In [3]:
data2

,sentence,predict
0,아 서울 중앙 지검 첨단 범죄 수사 일부 이희나 수사관입니다. 본 사건 조사는 법원...,2
1,네 네 OOO씨 지금 제가 본인 확인 한번 할께요. 생년월일 앞에 여섯 자리만 말...,2
2,네 성함은 OOO님 맞으시죠? 네 네 본인 김성오 일당 잘 알고 있으십니까? 아니요,2
3,전혀 모른다고 진술 하셨는데 맞습니까? 네 네 이천 십 칠년이 월 삼 일 경기도 ...,2
4,이 두개 계좌 계설 사실에 대해서 전혀 모르신다는 말씀 이시죠? 네 네 본인은 네...,2
...,...,...
1495,네. 근데 어딜 가야하나요? 제가? 아 어딜 가는게 아니라 제가 사건도 내용을 좀...,2
1496,그때 제가 절차 다 질문 드리면 본인께서는 아시는 대로 진술을 해주시면 되는겁니다...,2
1497,사건고지 좀 더 해 드리겠습니다. 또한 이번에 검거된 범인들 다수가 카드사 보험사...,2
1498,김태환이란 사람은 농협에서 8년 간 과장으로 근무한 경력이 확인되었어요. 현재 이...,2


In [4]:
import random
from typing import Dict, List
import MeCab

# Mecab 형태소 분석기 초기화
mecab = MeCab.Tagger()

In [5]:
similar_words_dict = {
    '계좌': ['은행 계좌', '계정', '입출금 계좌', '은행 계정'],
    '이체': ['송금', '전송', '이전', '이체하기'],
    '금융감독원': ['금감원', '금융감독기구', '금융감독청', '금융감독 기관'],
    '신분증': ['주민등록증', '신분 증명서', 'ID', '신분증', '식별증'],
    '인증': ['확인', '검증', '인증 절차', '인증하기'],
    '피싱': ['사기', '피싱 공격', '속임수', '보이스피싱'],
    '보안': ['안전', '보안 조치', '보호', '보안 유지'],
    '비밀번호': ['암호', '패스워드', '보안 코드', '비번'],
    '출금': ['인출', '빼내기', '자금 인출', '출금하기'],
    '입금': ['예금', '돈 입금', '자금 입금', '입금하기'],
    '계정': ['아이디', '사용자 계정', '로그인 계정', '사용자 ID'],
    '사기': ['사기 행위', '사기죄', '사기 범죄', '속임수'],
    '해킹': ['침입', '크래킹', '보안 해제', '해킹 공격'],
    '개인정보': ['개인 식별 정보', '개인 데이터', '민감 정보', '개인 정보'],
    '송금 요청': ['이체 요구', '송금 지시', '자금 이체 요청', '송금 요구'],
    '긴급 연락': ['긴급 통화', '긴급 소식', '긴급 연락처', '긴급 연락 사항'],
    '보안 카드': ['보안 토큰', '안전 카드', '보안 인증 카드', '보안 인증 토큰'],
    '조작': ['조종', '조정', '속임', '위조'],
    '유출': ['노출', '폭로', '누설', '정보 유출'],
    '피해 보고': ['피해 신고', '사기 신고', '피해 접수', '사고 신고'],
    '가짜': ['위조', '모조', '거짓', '가짜 정보'],
    '대포통장': ['위장통장', '타인명의 통장', '불법통장', '대포 계좌'],
    '돈': ['금액', '자금', '화폐', '자본'],
    '녹취': ['음성 기록', '통화 녹음', '녹음 파일', '음성 녹음'],
    '사실': ['진실', '현실', '실제', '진실 여부'],
    '판매': ['매매', '거래', '판매중', '판매하기'],
    '통화': ['전화', '통신', '대화', '전화하기'],
    '번호': ['숫자', '식별번호', '연락처', '전화번호'],
    '수사': ['조사', '검사', '수사 진행', '경찰 수사'],
    '거래': ['매매', '교환', '거래 내역', '거래하기'],
    '자금': ['돈', '금액', '자본', '재정'],
    '도용': ['훔치기', '사용허가 없음', '정보 도난', '정보 탈취'],
    '진술': ['설명', '증언', '발언', '진술하기'],
    '연루': ['관련됨', '개입', '참여', '연관됨'],
    '사건': ['사고', '일', '상황', '사태'],
    '범죄': ['죄', '불법 행위', '범죄 행위', '불법적 행동'],
    '유출': ['누설', '공개', '방출', '정보 누설'],
    '압수': ['몰수', '차지', '확보', '압수하기'],
    '피해자': ['손해를 본 사람', '피해를 입은 사람', '피해 당사자', '손해 본 이'],
}





In [6]:
def augment_sentence(sentence: str, similar_words_dict: Dict[str, List[str]]) -> str:
    parsed = mecab.parse(sentence)
    words = [line.split('\t')[0] for line in parsed.split('\n') if line.strip() != 'EOS' and line.strip() != '']
    

    word_positions = []
    start = 0
    for word in words:
        start = sentence.find(word, start)
        word_positions.append((start, word))
        start += len(word)

    augmented_sentence = sentence


    for start, word in reversed(word_positions):
        found_similar = False

        for key in similar_words_dict.keys():
            if word == key:
               
                augmented_word = random.choice(similar_words_dict[key])
                augmented_sentence = augmented_sentence[:start] + augmented_word + augmented_sentence[start+len(word):]
                found_similar = True
                break
        
    return augmented_sentence

In [7]:
X1=pd.DataFrame(index=range(data1.shape[0]),columns=['sentence','predict'])

In [8]:
for i in range(data1.shape[0]):
    A=augment_sentence(data1.iloc[i,0],similar_words_dict)
    if A==data1.iloc[i,0]:
        continue
    else:
        X1.iloc[i,0]=A

In [9]:
X1

,sentence,predict
0,여보세요 네 님 예 예 어저께 전화하기했던 최영환입니다,NaN
1,아 예 예 어떻게 가진행 결과 안내차 오늘 아침에 전화 드린다고 제가 얘기를 했었...,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
1391,NaN,NaN
1392,NaN,NaN
1393,아 네 안녕하세요 저희 그 하나캐피탈에서 연락을 드렸습니다 아 그래요? 네네네 고...,NaN
1394,NaN,NaN


In [10]:
X1.isnull().sum()

sentence     992
predict     1396
dtype: int64

In [11]:
X1=pd.DataFrame(X1['sentence'].dropna(axis=0))

In [12]:
X1

,sentence
0,여보세요 네 님 예 예 어저께 전화하기했던 최영환입니다
1,아 예 예 어떻게 가진행 결과 안내차 오늘 아침에 전화 드린다고 제가 얘기를 했었...
10,위임하실 경우에는 방문이나 내방 없이 드리는 전화 받으시고 통화 녹음를 통해서 대...
26,다른거 필요한 건 없나요? 뭐 추가적인 접수는 없으세요 아 뭐 통장 식별번호는 안...
27,지금 다짜고짜 여쭤보면 어떡합니까? 고객님 신분 증명서도 접수도 안 해줬는데 그냥...
...,...
1385,네 속임수치지 마시고요 다른사람한테 피해주지 마시구요 그러다 벌받아요
1386,무슨소리하시는거죠? 지금 그 계정 제가 신고 해놨거든요? 네 입출금 계좌에 금액을...
1389,사장님께서 혹시 안쓰시는 입출금 계좌가 있으시면 저희가 월 400에서 450만원까...
1390,아... 금액은 만약에 하면 어떻게 받는거에요? 저희는 그 손님들 게임머니 펀드용...


In [13]:
ab_data1=pd.DataFrame(index=range(20000),columns=['sentence','predict'])

In [14]:
count=0
for i in range(int(20000/X1.shape[0])):
    for j in range(X1.shape[0]):
        A=augment_sentence(X1.iloc[j,0],similar_words_dict)
        if A==X1.iloc[j,0]:
            continue
        else:
            ab_data1.iloc[count,0]=A
            count=count+1
        

In [15]:
ab_data1.isnull().sum()

sentence     9661
predict     20000
dtype: int64

In [16]:
ab_data1=pd.DataFrame(ab_data1['sentence'].dropna(axis=0))

In [17]:
ab_data1

,sentence
0,위임하실 경우에는 방문이나 내방 없이 드리는 전화 받으시고 대화 녹음를 통해서 대...
1,다른거 필요한 건 없나요? 뭐 추가적인 접수는 없으세요 아 뭐 통장 식별숫자는 안...
2,지금 다짜고짜 여쭤보면 어떡합니까? 고객님 신분 증명서도 접수도 안 해줬는데 그냥...
3,예 차후에 변동 안 되세요 예 대출 승인 나셨을 경우에 대출금 송금 받아 보실 수...
4,수납 담당자님 사용자 계정 연락처는 바로 카카오톡으로 보내드릴거구요. 수납 가능하...
...,...
10334,원래는 먼저 넣는 거 아니에요. 근데 위임처리하실 경우에만 먼저 받는 거예요 아 ...
10335,아휴 그럼 어떻게 해요. 제가 금액 자금 입금 바로 쓸 수가 있는거지. 느낌이 완...
10336,고객님 연락이 너무 안되셔가지고 지금 아 네 제가 너무 바빠서요 아 지금은 좀 연...
10337,무슨소리하시는거죠? 지금 그 사용자 ID 제가 신고 해놨거든요? 네 입출금 입출금...


In [18]:
X2=pd.DataFrame(index=range(data2.shape[0]),columns=['sentence','predict'])

In [19]:
for i in range(data2.shape[0]):
    A=augment_sentence(data2.iloc[i,0],similar_words_dict)
    if A==data2.iloc[i,0]:
        continue
    else:
        X2.iloc[i,0]=A

In [20]:
X2

,sentence,predict
0,아 서울 중앙 지검 첨단 범죄 행위 조사 일부 이희나 수사관입니다. 본 사고 조사는...,NaN
1,NaN,NaN
2,NaN,NaN
3,전혀 모른다고 증언 하셨는데 맞습니까? 네 네 이천 십 칠년이 월 삼 일 경기도 ...,NaN
4,이 두개 은행 계좌 계설 현실에 대해서 전혀 모르신다는 말씀 이시죠? 네 네 본인...,NaN
...,...,...
1495,네. 근데 어딜 가야하나요? 제가? 아 어딜 가는게 아니라 제가 사고도 내용을 좀...,NaN
1496,그때 제가 절차 다 질문 드리면 본인께서는 아시는 대로 증언을 해주시면 되는겁니다...,NaN
1497,사태고지 좀 더 해 드리겠습니다. 또한 이번에 검거된 범인들 다수가 카드사 보험사...,NaN
1498,김태환이란 사람은 농협에서 8년 간 과장으로 근무한 경력이 확인되었어요. 현재 이...,NaN


In [21]:
X2=pd.DataFrame(X2['sentence'].dropna(axis=0))

In [22]:
ab_data2=pd.DataFrame(index=range(20000),columns=['sentence','predict'])

In [23]:
X2

,sentence
0,아 서울 중앙 지검 첨단 범죄 행위 조사 일부 이희나 수사관입니다. 본 사고 조사는...
3,전혀 모른다고 증언 하셨는데 맞습니까? 네 네 이천 십 칠년이 월 삼 일 경기도 ...
4,이 두개 은행 계좌 계설 현실에 대해서 전혀 모르신다는 말씀 이시죠? 네 네 본인...
5,네 이 상황은 전자 금융 거래법 위반 금융 죄 속임수 상황으로 다수의 손해를 본 ...
6,네 근데 이런 조사 과정에서 불법적으로 사용된 내용이 발견이 되신다면 본인 명의의...
...,...
1495,네. 근데 어딜 가야하나요? 제가? 아 어딜 가는게 아니라 제가 사고도 내용을 좀...
1496,그때 제가 절차 다 질문 드리면 본인께서는 아시는 대로 증언을 해주시면 되는겁니다...
1497,사태고지 좀 더 해 드리겠습니다. 또한 이번에 검거된 범인들 다수가 카드사 보험사...
1498,김태환이란 사람은 농협에서 8년 간 과장으로 근무한 경력이 확인되었어요. 현재 이...


In [24]:
count=0
for i in range(int(20000/X2.shape[0])):
    for j in range(X2.shape[0]):
        A=augment_sentence(X2.iloc[j,0],similar_words_dict)
        if A==X2.iloc[j,0]:
            continue
        else:
            ab_data2.iloc[count,0]=A
            count=count+1
      

In [25]:
ab_data2=pd.DataFrame(ab_data1['sentence'].dropna(axis=0))

In [26]:
ab_data1

,sentence
0,위임하실 경우에는 방문이나 내방 없이 드리는 전화 받으시고 대화 녹음를 통해서 대...
1,다른거 필요한 건 없나요? 뭐 추가적인 접수는 없으세요 아 뭐 통장 식별숫자는 안...
2,지금 다짜고짜 여쭤보면 어떡합니까? 고객님 신분 증명서도 접수도 안 해줬는데 그냥...
3,예 차후에 변동 안 되세요 예 대출 승인 나셨을 경우에 대출금 송금 받아 보실 수...
4,수납 담당자님 사용자 계정 연락처는 바로 카카오톡으로 보내드릴거구요. 수납 가능하...
...,...
10334,원래는 먼저 넣는 거 아니에요. 근데 위임처리하실 경우에만 먼저 받는 거예요 아 ...
10335,아휴 그럼 어떻게 해요. 제가 금액 자금 입금 바로 쓸 수가 있는거지. 느낌이 완...
10336,고객님 연락이 너무 안되셔가지고 지금 아 네 제가 너무 바빠서요 아 지금은 좀 연...
10337,무슨소리하시는거죠? 지금 그 사용자 ID 제가 신고 해놨거든요? 네 입출금 입출금...


In [27]:
ab_data2

,sentence
0,위임하실 경우에는 방문이나 내방 없이 드리는 전화 받으시고 대화 녹음를 통해서 대...
1,다른거 필요한 건 없나요? 뭐 추가적인 접수는 없으세요 아 뭐 통장 식별숫자는 안...
2,지금 다짜고짜 여쭤보면 어떡합니까? 고객님 신분 증명서도 접수도 안 해줬는데 그냥...
3,예 차후에 변동 안 되세요 예 대출 승인 나셨을 경우에 대출금 송금 받아 보실 수...
4,수납 담당자님 사용자 계정 연락처는 바로 카카오톡으로 보내드릴거구요. 수납 가능하...
...,...
10334,원래는 먼저 넣는 거 아니에요. 근데 위임처리하실 경우에만 먼저 받는 거예요 아 ...
10335,아휴 그럼 어떻게 해요. 제가 금액 자금 입금 바로 쓸 수가 있는거지. 느낌이 완...
10336,고객님 연락이 너무 안되셔가지고 지금 아 네 제가 너무 바빠서요 아 지금은 좀 연...
10337,무슨소리하시는거죠? 지금 그 사용자 ID 제가 신고 해놨거든요? 네 입출금 입출금...


In [28]:
Y1=pd.DataFrame(index=range(ab_data1.shape[0]),columns=['predict'])

In [29]:
Y1

,predict
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
10334,NaN
10335,NaN
10336,NaN
10337,NaN


In [30]:
Y1=Y1.fillna(1)

/var/folders/gb/hslcbzfs0bvcnj3zxtfrxt_r0000gn/T/ipykernel_5254/908238685.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y1=Y1.fillna(1)


In [31]:
data1_to_save=pd.concat([ab_data1,Y1],axis=1)

In [32]:
data1_to_save

,sentence,predict
0,위임하실 경우에는 방문이나 내방 없이 드리는 전화 받으시고 대화 녹음를 통해서 대...,1
1,다른거 필요한 건 없나요? 뭐 추가적인 접수는 없으세요 아 뭐 통장 식별숫자는 안...,1
2,지금 다짜고짜 여쭤보면 어떡합니까? 고객님 신분 증명서도 접수도 안 해줬는데 그냥...,1
3,예 차후에 변동 안 되세요 예 대출 승인 나셨을 경우에 대출금 송금 받아 보실 수...,1
4,수납 담당자님 사용자 계정 연락처는 바로 카카오톡으로 보내드릴거구요. 수납 가능하...,1
...,...,...
10334,원래는 먼저 넣는 거 아니에요. 근데 위임처리하실 경우에만 먼저 받는 거예요 아 ...,1
10335,아휴 그럼 어떻게 해요. 제가 금액 자금 입금 바로 쓸 수가 있는거지. 느낌이 완...,1
10336,고객님 연락이 너무 안되셔가지고 지금 아 네 제가 너무 바빠서요 아 지금은 좀 연...,1
10337,무슨소리하시는거죠? 지금 그 사용자 ID 제가 신고 해놨거든요? 네 입출금 입출금...,1


In [33]:
Y2=pd.DataFrame(index=range(ab_data2.shape[0]),columns=['predict'])

In [34]:
Y2=Y2.fillna(2)

/var/folders/gb/hslcbzfs0bvcnj3zxtfrxt_r0000gn/T/ipykernel_5254/1604059481.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y2=Y2.fillna(2)


In [35]:
data2_to_save=pd.concat([ab_data2,Y2],axis=1)

In [36]:
data2_to_save

,sentence,predict
0,위임하실 경우에는 방문이나 내방 없이 드리는 전화 받으시고 대화 녹음를 통해서 대...,2
1,다른거 필요한 건 없나요? 뭐 추가적인 접수는 없으세요 아 뭐 통장 식별숫자는 안...,2
2,지금 다짜고짜 여쭤보면 어떡합니까? 고객님 신분 증명서도 접수도 안 해줬는데 그냥...,2
3,예 차후에 변동 안 되세요 예 대출 승인 나셨을 경우에 대출금 송금 받아 보실 수...,2
4,수납 담당자님 사용자 계정 연락처는 바로 카카오톡으로 보내드릴거구요. 수납 가능하...,2
...,...,...
10334,원래는 먼저 넣는 거 아니에요. 근데 위임처리하실 경우에만 먼저 받는 거예요 아 ...,2
10335,아휴 그럼 어떻게 해요. 제가 금액 자금 입금 바로 쓸 수가 있는거지. 느낌이 완...,2
10336,고객님 연락이 너무 안되셔가지고 지금 아 네 제가 너무 바빠서요 아 지금은 좀 연...,2
10337,무슨소리하시는거죠? 지금 그 사용자 ID 제가 신고 해놨거든요? 네 입출금 입출금...,2


In [37]:
data1_to_save.to_csv("대출 사기형 보이스피싱(증강) 0519.csv")

In [38]:
data2_to_save.to_csv("기관 사칭형 보이스피싱(증강) 0519.csv")